In [1]:
//val action = "create"
//val action = "remove"
val action = "none"

action: String = "none"

In [2]:
// common imports
import $ivy.`com.amazonaws:aws-java-sdk:1.11.22`

import $ivy.$                                   

In [3]:
import ammonite.ops._

val stackName = "test"
/// parameters
// template is for us-east-1
// change images for different regions
val region = "us-east-1"
val imageId = "ami-10f53a06"
val instanceType = "t2.small"
val keyName = "atomic"
val template = pwd/up/'aws/"kubernetes.yml"

import ammonite.ops._


stackName: String = "test"
region: String = "us-east-1"
imageId: String = "ami-10f53a06"
instanceType: String = "t2.small"
keyName: String = "atomic"
template: ammonite.ops.Path = /Users/msciab/Dropbox/Work/MacBookPro/MicheleSciabarra/Mosaico/cloud/aws/kubernetes.yml

In [4]:
import com.amazonaws.services.cloudformation.model._
import collection.JavaConverters._

// create
def createRequest() = {
    import com.amazonaws.services.cloudformation._
    import com.amazonaws.services.cloudformation.model._
    val cfr = new CreateStackRequest()
    cfr.setStackName(stackName)
    val params =  Seq(
      new Parameter().withParameterKey("KeyName").withParameterValue(keyName),
      new Parameter().withParameterKey("InstanceType").withParameterValue(instanceType),
      new Parameter().withParameterKey("ImageId").withParameterValue(imageId)    
    ) 
    cfr.setParameters(params.asJava)
    cfr
}
// remove
def removeRequest() = 
    new DeleteStackRequest().withStackName(stackName)

import com.amazonaws.services.cloudformation.model._

import collection.JavaConverters._

// create

defined function createRequest
defined function removeRequest

In [5]:
import com.amazonaws.services._
import com.amazonaws.services.cloudformation._

val cf = new AmazonCloudFormationClient() 
def status = {
    val stack = cf.describeStacks
        .getStacks.asScala
        .filter( _.getStackName == stackName).headOption
    stack.map(_.getStackStatus)
}
def statusLoop {
    val st = status
    println(st)
    if(st.nonEmpty && st.get.endsWith("_IN_PROGRESS")) {
        Thread.sleep(1000)
        statusLoop
    }
}

import com.amazonaws.services._

import com.amazonaws.services.cloudformation._


cf: com.amazonaws.services.cloudformation.AmazonCloudFormationClient = com.amazonaws.services.cloudformation.AmazonCloudFormationClient@76b30007
defined function status
defined function statusLoop

## Creation

In [6]:
// read template
import scala.io._
import com.amazonaws.services.cloudformation.model._

if(action == "create") {
    val cfr = createRequest()
    val body = Source.fromFile(template.toString).getLines.mkString("\n")
    cfr.setTemplateBody(body)
    // run request
    val res = cf.createStack(cfr)
    statusLoop
}

import scala.io._

import com.amazonaws.services.cloudformation.model._



## Cleanup

In [7]:
if(action == "remove") { 
    cf.deleteStack(removeRequest())
    statusLoop
}


## Building the inventory

In [8]:
import com.amazonaws.services.ec2._
import com.amazonaws.regions._


val ec2 = new AmazonEC2Client()
val regions = Regions.fromName(region)
ec2.setRegion(Region.getRegion(regions))

import com.amazonaws.services.ec2._

import com.amazonaws.regions._



ec2: com.amazonaws.services.ec2.AmazonEC2Client = com.amazonaws.services.ec2.AmazonEC2Client@11d3e850
regions: com.amazonaws.regions.Regions = US_EAST_1

In [9]:
import collection.JavaConverters._

//val reservation = ec2.describeInstances.getReservations.asScala
val ips = for {
 reservation <- ec2.describeInstances.getReservations.asScala
 instance <- reservation.getInstances.asScala
} yield {
    instance.getPrivateIpAddress -> instance.getPublicIpAddress
}

import collection.JavaConverters._

//val reservation = ec2.describeInstances.getReservations.asScala

ips: collection.mutable.Buffer[(String, String)] = ArrayBuffer(
  ("10.0.0.24", "34.203.59.213"),
  ("10.0.0.171", "54.173.55.251"),
  ("10.0.0.227", "54.83.145.51")
)

In [12]:
val inventory = Seq(
    "[all:vars]",
    s"kube_master_host=${ips.head._2}",
    s"kube_master=${ips.head._1}",    
    "[aws]"
) ++ ips.map(_._2) ++
  Seq("[masters]", s"${ips.head._2} local_ip=${ips.head._1}") ++
  Seq("[nodes]") ++
  ips.tail.map(t => s"${t._2} local_ip=${t._1}")
val inv = pwd/up/'aws/"inventory"
rm! inv
write(inv, inventory.mkString("\n"))
show(inventory)

List(
  "[all:vars]",
  "kube_master_host=34.203.59.213",
  "kube_master=10.0.0.24",
  "[aws]",
  "34.203.59.213",
  "54.173.55.251",
  "54.83.145.51",
  "[masters]",
  "34.203.59.213 local_ip=10.0.0.24",
  "[nodes]",
  "54.173.55.251 local_ip=10.0.0.171",
  "54.83.145.51 local_ip=10.0.0.227"
)


inventory: Seq[String] = List(
  "[all:vars]",
  "kube_master_host=34.203.59.213",
  "kube_master=10.0.0.24",
  "[aws]",
  "34.203.59.213",
  "54.173.55.251",
  "54.83.145.51",
  "[masters]",
  "34.203.59.213 local_ip=10.0.0.24",
  "[nodes]",
  "54.173.55.251 local_ip=10.0.0.171",
...
inv: Path = /Users/msciab/Dropbox/Work/MacBookPro/MicheleSciabarra/Mosaico/cloud/aws/inventory